In [ ]:
def func_1(df):
    num = df['km_cluster'].unique()
    for x in range(num):
        tmp_df = df[df['km_cluster'] == [x]]
        tmp_df = tmp_df.drop('km_cluster',axis=1)

        col = tmp_df.columns[3:-2]
        data = tmp_df[col].values

        pca_transformed = pca.fit_transform(data)
        tmp_df['pca_x'] = pca_transformed[:,0]
        tmp_df['pca_y'] = pca_transformed[:,1]

        inertia = [] # 군집의 centre 와 sample 사이의 거리의 제곱의 합
        plt.figure(figsize=(10,6))
        for i in range(1, 10): # 군집 개수에 따른 
            kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, random_state=0)
            kmeans.fit(pca_transformed)
            inertia.append(kmeans.inertia_)
        plt.plot(range(1, 10), inertia, marker='o')
        plt.title('The Elbow Method')
        plt.xlabel('Number of clusters')
        plt.ylabel('Inertia')
        plt.show()

        kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, random_state=0)
        # label
        label_1 = kmeans.fit(pca_transformed)
        # Getting unique labels
        u_labels = np.unique(label_1)

        tmp_df['km_cluster_1'] = kmeans.labels_
        tmp_df.groupby('km_cluster_1').count()

        tmp_df['pca_x'] = pca_transformed[:,0]
        tmp_df['pca_y'] = pca_transformed[:,1]

        # 클러스터별 인덱스 추출
        marker_n = len(tmp_df['km_cluster_1'].unique())
        markers = []
        for i in range(marker_n):
            marker = tmp_df[tmp_df['km_cluster_1'] == i].index
            markers.append(marker)

        m = ['o','s','^','v','>']
        for i in range(marker_n):
            plt.scatter(x = tmp_df.loc[markers[i], 'pca_x'], y = tmp_df.loc[markers[i], 'pca_y'], marker = m[i])

        plt.xlabel('PCA1')
        plt.ylabel('PCA2')
        plt.title('Clusters Visualization by 2 PCA Components')
        plt.legend(['cluster0', 'cluster1','cluster2'])
        plt.show()

        # visualizer 생성
        visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
        # 생성 된 visualizer에 데이터 입력 
        visualizer.fit(pca_transformed)      
        visualizer.show()

        score = silhouette_score(data,tmp_df['km_cluster_1'])
        print('기본 데이터 셋 Silhouette Score : {0: .3f}'.format(score))